# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/cheng/spark-2.3.0-bin-hadoop2.7')
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_project').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [4]:
data = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
data.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [7]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [9]:
cruise_cat_indexer = StringIndexer(inputCol='Cruise_line',outputCol='Cruise_cat_index')
indexed = cruise_cat_indexer.fit(data).transform(data)
indexed.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_cat_index=16.0)

In [10]:
cruise_cat_encoder = OneHotEncoder(inputCol='Cruise_cat_index',outputCol='Cruise_cat_vec')
encoded = cruise_cat_encoder.transform(indexed)
encoded.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_cat_index=16.0, Cruise_cat_vec=SparseVector(19, {16: 1.0}))

In [11]:
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins',
                                       'passenger_density','Cruise_cat_vec'],outputCol='features')

In [12]:
output = assembler.transform(encoded)
output.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_cat_index=16.0, Cruise_cat_vec=SparseVector(19, {16: 1.0}), features=SparseVector(25, {0: 6.0, 1: 30.277, 2: 6.94, 3: 5.94, 4: 3.55, 5: 42.64, 22: 1.0}))

In [13]:
final_data = output.select("features",'crew')

In [14]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [15]:
lr = LinearRegression(labelCol='crew',featuresCol='features')

In [16]:
lr_model = lr.fit(train_data)

In [17]:
print("Coefficients: {} Intercept: {}".format(lr_model.coefficients, lr_model.intercept))

Coefficients: [0.003332136608224832,0.009468788441406552,-0.1020027335252254,0.3887004197663304,0.8130270305426731,0.009307997792235978,-0.9628193942835772,0.607186828836825,0.3299562559672139,-0.21741338634290153,0.8705105220428768,-0.18507237711836805,0.9119000520793636,0.13873636703589276,1.0613450745163997,0.36235286634018965,0.9534292040021761,0.4136320574734811,0.8436164077173216,0.6114749442385433,0.019978483778314213,0.36920347025430794,0.3365270797802936,1.0731279452214904,0.5166200051912918] Intercept: -1.9773269141044778


In [18]:
training_summary = lr_model.summary
print("RMSE: {}".format(training_summary.rootMeanSquaredError))
print("r2: {}".format(training_summary.r2))

RMSE: 0.8598306031887373
r2: 0.9437084251760924


In [19]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               105|
|   mean|  7.85780952380953|
| stddev|3.6414066335775073|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [20]:
test_results = lr_model.evaluate(test_data)

In [21]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("MAE: {}".format(test_results.meanAbsoluteError))
print("r2: {}".format(test_results.r2))

RMSE: 0.7285614982125919
MSE: 0.5308018566777766
MAE: 0.5101287269659852
r2: 0.9485499660722019
